In a previous post, I covered the fundamental concepts of window functions, with a focus on window frames (ROWS, RANGE, GROUPS) and their syntax. Now, let’s delve into the various functions available within SQL window functions. These functions can be categorized as follows:

Aggregate functions:

MIN(): Computes the minimum value within a partition.
MAX(): Calculates the maximum value within a partition.
COUNT(): Counts the number of rows in a partition.
SUM(): Computes the sum of values within a partition.
AVG(): Calculates the average value within a partition.

Ranking functions:

ROW_NUMBER(): Assigns a unique sequential number to each row in a result set, starting from 1.
RANK(): Assigns ranks based on specified criteria, with tied values receiving the same rank.
DENSE_RANK(): Similar to RANK(), but without gaps in rankings.

Distribution functions:

PERCENT_RANK(): Computes the relative rank of each row within a partition.
CUME_DIST(): Calculates the cumulative distribution of values within a partition.

Analytic functions:

LEAD(): Retrieves the value of a subsequent row within a partition.
LAG(): Retrieves the value of a preceding row within a partition.
NTILE(): Divides rows into equal-sized buckets.
FIRST_VALUE(): Retrieves the first value within a partition.
LAST_VALUE(): Retrieves the last value within a partition.
NTH_VALUE(): Retrieves the nth value within a partition.
As we proceed, keep in mind the syntax of window functions.
'''
SELECT <column_1>, <column_2>,
 <window_function> OVER (
 PARTITION BY <...>
 ORDER BY <...>
 <window_frame>) <window_column_alias>
FROM <table_name>;
'''

In [1]:
import sqlite3
import pandas as pd
from tabulate import tabulate

# create a sqlite database with name test_sqlite.db
db_path = "data/db_test.db"

# path of spreadsheet file
excel_file = 'data/test_data.xlsx'
# read spreadsheet data and put to dataframe
test_data = pd.read_excel(excel_file, sheet_name='test', header=0)
table_test = """
    CREATE TABLE test (city TEXT, id INTEGER, sold INTEGER, month INTEGER)
    """

with sqlite3.connect(db_path) as con:
    # delete the table if it exist
    con.execute( "DROP TABLE IF EXISTS test;")
    # execute these commands to create database tables
    con.execute(table_test)
    test_data.to_sql('test', con=con, if_exists='append',index=False)

First, let’s explore Aggregate Functions. These functions are fundamental and widely used in various programming languages. Here’s a breakdown of each function:

MIN(): Calculates the minimum value within the specified window frame.
MAX(): Determines the maximum value within the window frame.
COUNT(): Counts the number of rows within the window frame.
SUM(): Computes the sum of values within the window frame.
AVG(): Calculates the average value for the rows within the window frame.
Now, let’s take a look at an example code snippet that demonstrates the results of these aggregate functions. Consider the initial row of the ‘Paris’ partition. Due to the window frame defined as ‘ROWS BETWEEN 1 PRECEDING AND 2 FOLLOWING,’ the list of values within this frame is [300, 500, 200]. Based on this list, we can compute the following statistics:

Minimum (MIN): 200
Maximum (MAX): 500
Count: 3
Sum: 1000
Average: 333.333 (repeating)

In [4]:
# Aggregate functions 
cols = ['city', 'id','sold','month',"min_rows",'max_rows','count_rows','sum_rows','average_rows']
cmd3 = """
SELECT city,id,sold,month,
MIN(sold) OVER (
    PARTITION BY city
    ORDER BY month
    ROWS BETWEEN 1 PRECEDING AND 2 FOLLOWING)""" +cols[4]+""",
MAX(sold) OVER (
    PARTITION BY city
    ORDER BY month
    ROWS BETWEEN 1 PRECEDING AND 2 FOLLOWING)""" +cols[5]+""",
COUNT(sold) OVER (
    PARTITION BY city
    ORDER BY month
    ROWS BETWEEN 1 PRECEDING AND 2 FOLLOWING)""" +cols[6]+""",
SUM(sold) OVER (
    PARTITION BY city
    ORDER BY month
    ROWS BETWEEN 1 PRECEDING AND 2 FOLLOWING)""" +cols[7]+""",
AVG(sold) OVER (
    PARTITION BY city
    ORDER BY month
    ROWS BETWEEN 1 PRECEDING AND 2 FOLLOWING)""" +cols[8]+"""
FROM tb
ORDER BY city,month
"""
# print (cmd3)
with sqlite3.connect(db_path) as con:
    re=con.execute(cmd3)
print(tabulate(re, headers=cols, tablefmt='psql'))


SELECT city,id,sold,month,
MIN(sold) OVER (
    PARTITION BY city
    ORDER BY month
    ROWS BETWEEN 1 PRECEDING AND 2 FOLLOWING)min_rows,
MAX(sold) OVER (
    PARTITION BY city
    ORDER BY month
    ROWS BETWEEN 1 PRECEDING AND 2 FOLLOWING)max_rows,
COUNT(sold) OVER (
    PARTITION BY city
    ORDER BY month
    ROWS BETWEEN 1 PRECEDING AND 2 FOLLOWING)count_rows,
SUM(sold) OVER (
    PARTITION BY city
    ORDER BY month
    ROWS BETWEEN 1 PRECEDING AND 2 FOLLOWING)sum_rows,
AVG(sold) OVER (
    PARTITION BY city
    ORDER BY month
    ROWS BETWEEN 1 PRECEDING AND 2 FOLLOWING)average_rows
FROM tb
ORDER BY city,month

+--------+------+--------+---------+------------+------------+--------------+------------+----------------+
| city   |   id |   sold |   month |   min_rows |   max_rows |   count_rows |   sum_rows |   average_rows |
|--------+------+--------+---------+------------+------------+--------------+------------+----------------|
| London |   30 |    200 |       5 |        100

Ranking functions assign a ranking value to each row within a partition. These functions operate without limitations on the window frame (which includes ROWs, RANGE, and GROUPS). There are three primary ranking functions:

ROW_NUMBER(): This function provides a unique ranking within a partition, with no gaps between ranks. Tied values receive different rankings.
RANK(): Within a partition, this function assigns ranks, allowing for gaps. Tied values receive the same ranking.
DENSE_RANK(): Similar to ROW_NUMBER(), this function assigns rankings within a partition, but without gaps. Tied values also receive the same ranking.
If you find the definitions confusing, take a look at the image below, which displays the result of an SQL command:

'''

SELECT city,id,sold,month,
ROW_NUMBER() OVER (
    ORDER BY month
    ROWS BETWEEN 1 PRECEDING AND 2 FOLLOWING) row_numbers_rows,
RANK() OVER (
    ORDER BY month
    ROWS BETWEEN 1 PRECEDING AND 2 FOLLOWING) rank_rows,
DENSE_RANK() OVER (
    ORDER BY month
    ROWS BETWEEN 1 PRECEDING AND 2 FOLLOWING) dense_rank_rows
FROM tb

'''

The first table displays the result of the ROW_NUMBER function. Each row is assigned a unique sequential number starting from 1, based on the specified ORDER BY expression. Notably, even though there are two rows with a value of 1 in the ‘month’ column, they receive distinct rankings (1 and 2) due to the different ranking approach for tied values. Consequently, the ranking provided by ROW_NUMBER is unique.

The second table represents the outcome of the RANK function. In this case, the second row receives a ranking of 1, and the third row is ranked 3. The gap between these rankings occurs for two reasons:
    Firstly, RANK assigns the same ranking to tied values. Since the second row’s ‘month’ column value matches the first row, it shares the ranking of 1 with the first row.
    Secondly, the third row has a distinct value (2) compared to the first two rows. As a result, its ranking follows the order of appearance in the table, resulting in a ranking of 3.

The last table shows the result of the DENSE_RANK function. Here, tied values receive the same ranking. Consequently, the first two rows share a ranking of 1 because their ‘month’ values are both 1. Unlike RANK, there are no gaps in the rankings. Therefore, the third row’s ranking is 2 (instead of 3, as in the case of RANK).

![Alt text](images/fucntion_row_number_rank_dense_rank.png)

*source:  me*



In [7]:
 
cols = ['city', 'id','sold','month',"row_numbers",'rank','dense_rank']
cmd3 = """
SELECT city,id,sold,month,
ROW_NUMBER() OVER ()""" +cols[4]+""",
RANK() OVER (
    ORDER BY month)""" +cols[5]+""",
DENSE_RANK() OVER (
    ORDER BY month)""" +cols[6]+"""
FROM tb
"""
# print (cmd3)
with sqlite3.connect(db_path) as con:
    re=con.execute(cmd3)
print(tabulate(re, headers=cols, tablefmt='psql'))



SELECT city,id,sold,month,
ROW_NUMBER() OVER ()row_numbers,
RANK() OVER (
    ORDER BY month)rank,
DENSE_RANK() OVER (
    ORDER BY month)dense_rank
FROM tb


+--------+------+--------+---------+---------------+--------+--------------+
| city   |   id |   sold |   month |   row_numbers |   rank |   dense_rank |
|--------+------+--------+---------+---------------+--------+--------------|
| Paris  |   10 |    300 |       1 |             1 |      1 |            1 |
| Rome   |   40 |    200 |       1 |             2 |      1 |            1 |
| Paris  |   20 |    500 |       2 |             3 |      3 |            2 |
| Rome   |   10 |    100 |       3 |             4 |      4 |            3 |
| Rome   |   20 |    100 |       4 |             5 |      5 |            4 |
| Paris  |   20 |    200 |       4 |             6 |      5 |            4 |
| Paris  |   30 |    300 |       5 |             7 |      7 |            5 |
| Rome   |   40 |    200 |       5 |             8 |      7 |         

Next paraphrap, we delve to understand Distribution Functions. There are two functions that are blonged to the category.

PERCENT_RANK(): the percentile ranking number of a row—a value in [0, 1] interval:
(rank - 1) / (total number of rows - 1)

CUME_DIST(): the cumulative distribution of a value within a group of values, i.e., the number of rows with values less than or equal to the current row’s value divided by the total number of rows;a value in (0, 1] interval

The syntax of these fucntions as below

'''
SELECT city,id,sold,month,
PERCENT_RANK() OVER (
    ORDER BY sold) AS percent_rank,
CUME_DIST() OVER (
    ORDER BY sold) AS cume_dist
FROM tb
ORDER BY sold
'''

In the following image, we explore the concept of Distribution Functions. These functions involve ranking and are accompanied by a newly added ‘ranking’ column for clarity.

PERCENT_RANK(): In the first table, we examine the results of the PERCENT_RANK() function. Notably, the ‘percent_rank’ values for the initial three rows are identical due to their shared ranking. Specifically, ‘percent_rank’ is calculated as (1 (current ranking) - 1) / (11 (total rows of partition) - 1), resulting in a value of 0.

CUME_DIST(): Moving to the second table, we observe the outcomes of CUME_DIST(). To enhance understanding, an additional column called ‘number_of_rows_less_or_equal_current_ranking’ has been appended. Each row in this column represents the count of rows in the ‘ranking’ column with values less than or equal to the current value of ‘ranking’. For instance:  
    - The first three rows have a count of 3 because only 3 rows in the ‘ranking’ column have a value of 1 (matching the current ranking).   
    - The subsequent four rows have a count of 7. This is due to 3 occurrences of the value 1 and 4 occurrences of the value 4, both satisfying the condition of being less than or equal to the current ranking of 4.

CUME_DIST() calculates the number of rows with values less than or equal to the current ranking, divided by the total number of rows in a partition. The resulting values are presented in the second table, with the ‘cume_dist’ column derived from a straightforward calculation involving the other two columns.

![Alt text](images/function_percent_rank_cume_dist.png)

*source:  me*

In [6]:
 
cols = ['city', 'id','sold','month',"percent_rank",'cume_dist']
cmd3 = """
SELECT city,id,sold,month,
PERCENT_RANK() OVER (
    ORDER BY sold) AS """ +cols[4]+""",
CUME_DIST() OVER (
    ORDER BY sold) AS """ +cols[5]+"""
FROM tb
ORDER BY sold
"""
# print (cmd3)
with sqlite3.connect(db_path) as con:
    re=con.execute(cmd3)
print(tabulate(re, headers=cols, tablefmt='psql'))


SELECT city,id,sold,month,
PERCENT_RANK() OVER (
    ORDER BY sold) ASpercent_rank,
CUME_DIST() OVER (
    ORDER BY sold) AScume_dist
FROM tb
ORDER BY sold

+--------+------+--------+---------+----------------+-------------+
| city   |   id |   sold |   month |   percent_rank |   cume_dist |
|--------+------+--------+---------+----------------+-------------|
| Rome   |   10 |    100 |       3 |            0   |    0.272727 |
| Rome   |   20 |    100 |       4 |            0   |    0.272727 |
| London |   20 |    100 |       6 |            0   |    0.272727 |
| Rome   |   40 |    200 |       1 |            0.3 |    0.636364 |
| Paris  |   20 |    200 |       4 |            0.3 |    0.636364 |
| Rome   |   40 |    200 |       5 |            0.3 |    0.636364 |
| London |   30 |    200 |       5 |            0.3 |    0.636364 |
| Paris  |   10 |    300 |       1 |            0.7 |    0.909091 |
| Paris  |   30 |    300 |       5 |            0.7 |    0.909091 |
| Rome   |   10 |    300 |

Lastly, we dig to Analytic Fucntions. Analytic functions are powerful tools in SQL that allow us to perform calculations across rows in a result set. Among these functions, LEAD and LAG stand out. Let’s explore them:
LEAD(expression, offset, default): returns the values for a row at a given offset below the current row in the partition
LAG(expression, offset, default): is opposite with LEAD. It returns the values for a row at given offset abvove the current row in the partition.
in both fucntions, we have some arguments:
    - expression: the target column that the function operates on
    - offset: The number of rows forwards/backwards from curent row. Default value is 1.
    - default: if the "offset" goes beyond the range of the partition, then the function returns 'default'. if we do not specify 'default', NULL is returned. 

These functions require an ORDER BY clause to determine the row order
Syntax for the fucntions as below:
'''
SELECT city,id,month,sold,
LEAD(sold,2,0) OVER (
    ORDER BY month) AS lead,
LAG(sold,2,0) OVER (
    ORDER BY month) AS lag
FROM tb
ORDER BY month
'''


In our examination, we closely scrutinize the outcomes of the LEAD and LAG functions. Here’s what we observe:

LEAD Result:

The first table displays the results of the LEAD function.
Consider the value in the first row of the ‘lead’ column, which is 500.
This value corresponds to the third row of the ‘sold’ column.
How? By adding the current row position (1) in the ‘lead’ column to the offset (2), we get the position (3) in the ‘sold’ column.
The last two rows of the ‘lead’ column have a value of 0 (default).
Why? Because when we calculate the position (10 or 11) in the ‘lead’ column plus the offset (2), it exceeds the total number of rows (11).

LAG Result:

The second table presents the results of the LAG function.
Focus on the value in the last row of the ‘lag’ column, which is 200.
This value corresponds to the ninth row of the ‘sold’ column.
How? By subtracting the offset (2) from the current row position (11), we arrive at the position (9) in the ‘sold’ column.

In summary, both functions provide valuable insights by referencing neighboring rows within the partition.

![Alt text](images/fucntion_lead_lag.png)

*source:  me*

In [5]:
# Lead and Lag functions
cols = ['city', 'id','month','sold',"lead",'lag']
cmd3 = """
SELECT city,id,month,sold,
LEAD(sold,2,0) OVER (
    ORDER BY month) AS """ +cols[4]+""",
LAG(sold,2,0) OVER (
    ORDER BY month) AS """ +cols[5]+"""
FROM tb
ORDER BY month
"""
print (cmd3)
with sqlite3.connect(db_path) as con:
    re=con.execute(cmd3)
print(tabulate(re, headers=cols, tablefmt='psql'))


SELECT city,id,month,sold,
LEAD(sold,2,0) OVER (
    ORDER BY month) AS lead,
LAG(sold,2,0) OVER (
    ORDER BY month) AS lag
FROM tb
ORDER BY month

+--------+------+---------+--------+--------+-------+
| city   |   id |   month |   sold |   lead |   lag |
|--------+------+---------+--------+--------+-------|
| Paris  |   10 |       1 |    300 |    500 |     0 |
| Rome   |   40 |       1 |    200 |    100 |     0 |
| Paris  |   20 |       2 |    500 |    100 |   300 |
| Rome   |   10 |       3 |    100 |    200 |   200 |
| Rome   |   20 |       4 |    100 |    300 |   500 |
| Paris  |   20 |       4 |    200 |    200 |   100 |
| Paris  |   30 |       5 |    300 |    200 |   100 |
| Rome   |   40 |       5 |    200 |    100 |   200 |
| London |   30 |       5 |    200 |    300 |   300 |
| London |   20 |       6 |    100 |      0 |   200 |
| Rome   |   10 |       6 |    300 |      0 |   200 |
+--------+------+---------+--------+--------+-------+


Next, we explore at NTILE function. The function divied rows within a partition as equally as possible into n groups, and assign each row its group number. Let look at the example after run two querries with n of NTILE is 4 and 7 respectively. Remember that the ORDER BY clause plays a crucial role in determining the logical order of rows within each partition.
'''
SELECT city,id,month,sold,
NTILE(2) OVER (
    ORDER BY month) AS ntile_2,
NTILE(3) OVER (
    ORDER BY month) AS ntile_3
FROM tb
ORDER BY month
'''

As description of NTILE function, We start with a total of 11 rows in our table (due to no PARTITION BY clause).
We divide these rows into 4 buckets (where n is 4).
The quotient is 2, and the remainder is 3.
The first 3 groups (based on the remainder) each contain 3 rows (due to 2 (quotient) + 1).
The last group has 2 rows (based on the quotient).
The value for each group ranges from 1 to 4, respecting the order of the groups.

Next example with NTILE(7), we have 11 rows in our table (no PARTITION BY).
Dividing by 7 (where n is 7), we get a quotient of 1 and a remainder of 4.
The first 4 groups (based on the remainder) each contain 2 rows (due to 1 (quotient) + 1).
The last 3 groups have 1 row each (based on the quotient).
The value for each group ranges from 1 to 7, following the order of the groups.

![Alt text](images/fucntion_ntile.png)

*source:  me*

In [6]:
# NTILE function
cols = ['city', 'id','month','sold',"ntile_3","ntile_7"]
cmd3 = """
SELECT city,id,month,sold,
NTILE(4) OVER (
    ORDER BY month) AS """ +cols[4]+""",
NTILE(7) OVER (
    ORDER BY month) AS """ +cols[5]+"""
FROM tb
ORDER BY month
"""

print (cmd3)
with sqlite3.connect(db_path) as con:
    re=con.execute(cmd3)
print(tabulate(re, headers=cols, tablefmt='psql'))


SELECT city,id,month,sold,
NTILE(4) OVER (
    ORDER BY month) AS ntile_3,
NTILE(7) OVER (
    ORDER BY month) AS ntile_7
FROM tb
ORDER BY month

+--------+------+---------+--------+-----------+-----------+
| city   |   id |   month |   sold |   ntile_3 |   ntile_7 |
|--------+------+---------+--------+-----------+-----------|
| Paris  |   10 |       1 |    300 |         1 |         1 |
| Rome   |   40 |       1 |    200 |         1 |         1 |
| Paris  |   20 |       2 |    500 |         1 |         2 |
| Rome   |   10 |       3 |    100 |         2 |         2 |
| Rome   |   20 |       4 |    100 |         2 |         3 |
| Paris  |   20 |       4 |    200 |         2 |         3 |
| Paris  |   30 |       5 |    300 |         3 |         4 |
| Rome   |   40 |       5 |    200 |         3 |         4 |
| London |   30 |       5 |    200 |         3 |         5 |
| London |   20 |       6 |    100 |         4 |         6 |
| Rome   |   10 |       6 |    300 |         4 |         7 |

We went through most of the functions of window function, there are only 3 last functions FIRST_VALUE, LAST_VALUE, NTH_VALUE.  
FIRST_VALUE(expr) − the value for the first row within the window frame
LAST_VALUE(expr) − the value for the last row within the window frame
NTH_VALUE(expr, n) − the value for the n-th row within the window frame; n must be an integer. If the n-th row does not exist, the function returns NULL

These functions accept ORDER BY and window frame, but does not require. With only ORDER BY, LAST_VALUE retunrs the value for the current row.
Syntax of these function with PARTITION BY, ORDER BY and Window frame.
'''
SELECT city,id,month,sold,
FIRST_VALUE(sold) OVER (
    PARTITION BY city
    ORDER BY month
    ROWS BETWEEN 1 PRECEDING AND 2 FOLLOWING) AS first_value,
LAST_VALUE(sold) OVER (
    PARTITION BY city
    ORDER BY month
    ROWS BETWEEN 1 PRECEDING AND 2 FOLLOWING) AS last_value,
NTH_VALUE(sold,2) OVER (
    PARTITION BY city
    ORDER BY month
    ROWS BETWEEN 1 PRECEDING AND 2 FOLLOWING) AS nth_value
FROM tb
ORDER BY city,month
'''

Consider the following tables, which showcase the behavior of these window functions. If we look at first row in 'Paris' partition, then we have a list of value [300,500,200] because window frame is "ROWS BETWEEN 1 PRECEDING AND 2 FOLLOWING". Therefore, the value of first_value for the current row should be 300, the first value of the list. Similarly, last_value and 2nd_value column for current row should be 200 and 500 respectively. 


![Alt text](images/fucntion_first_last_nth.png)

*source:  me*

In [46]:
# First value, Last value and Nth value functions
cols = ['city', 'id','month','sold',"first_value","last_value","second_value"]
cmd3 = """
SELECT city,id,month,sold,
FIRST_VALUE(sold) OVER (
    PARTITION BY city
    ORDER BY month
    ROWS BETWEEN 1 PRECEDING AND 2 FOLLOWING) AS """ +cols[4]+""",
LAST_VALUE(sold) OVER (
    PARTITION BY city
    ROWS BETWEEN 1 PRECEDING AND 2 FOLLOWING) AS """ +cols[5]+""",
NTH_VALUE(sold,2) OVER (
    PARTITION BY city
    ORDER BY month
    ROWS BETWEEN 1 PRECEDING AND 2 FOLLOWING) AS """ +cols[6]+"""
FROM tb
ORDER BY city,month
"""
# print (cmd3)
with sqlite3.connect(db_path) as con:
    re=con.execute(cmd3)
print(tabulate(re, headers=cols, tablefmt='psql'))

+--------+------+---------+--------+---------------+--------------+----------------+
| city   |   id |   month |   sold |   first_value |   last_value |   second_value |
|--------+------+---------+--------+---------------+--------------+----------------|
| London |   30 |       5 |    200 |           200 |          100 |            100 |
| London |   20 |       6 |    100 |           200 |          100 |            100 |
| Paris  |   10 |       1 |    300 |           300 |          200 |            500 |
| Paris  |   20 |       2 |    500 |           300 |          300 |            500 |
| Paris  |   20 |       4 |    200 |           500 |          300 |            500 |
| Paris  |   30 |       5 |    300 |           200 |          300 |            500 |
| Rome   |   40 |       1 |    200 |           200 |          100 |            100 |
| Rome   |   10 |       3 |    100 |           200 |          200 |            100 |
| Rome   |   20 |       4 |    100 |           100 |          300

In conclusion, this post has covered all the essential window functions. I hope it provides some assistance in your journey to learn SQL. I uploaded code and documents here.

References:
Window functions cheat sheet: https://learnsql.com/blog/sql-window-functions-cheat-sheet/Window_Functions_Cheat_Sheet_Letter.pdf